In [ ]:
import numpy as np
from numpy import pi as π
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import tqdm
import firedrake
from firedrake import (
    exp, max_value, Constant, inner, grad, dx as dζ, ds, dS, avg, jump
)
import irksome
from irksome import Dt

In [ ]:
nx = 32
mesh = firedrake.UnitIntervalMesh(nx)

In [ ]:
cg = firedrake.FiniteElement("CG", "interval", 1)
dg = firedrake.FiniteElement("DG", "interval", 0)
r = firedrake.FiniteElement("R", "interval", 0)

In [ ]:
R = firedrake.FunctionSpace(mesh, r)
Q = firedrake.FunctionSpace(mesh, cg)

h = firedrake.Function(R).assign(1.0)

E = firedrake.Function(Q).assign(-1.0)

In [ ]:
Z = Q * R
z = firedrake.Function(Z)

z.subfunctions[0].assign(E)
z.subfunctions[1].assign(h)

For this 1D problem, the thickness is just a scalar variable.
It evolves in time by the difference of accumulation and ablation.

In [ ]:
def thickness_form(**kwargs):
    field_names = ("thickness", "accumulation", "ablation", "test_function")
    h, a, b, η = map(kwargs.get, field_names)
    return (Dt(h) - a + b) * η * dζ

We're going to work primarily with the enthalpy or energy density rather than the temperature.
But we still need to be able to compute the temperature because the diffusive heat flux is proportional to the temperature gradient:
$$F = -k\nabla T.$$
For temperature values below the melting point,
$$E = \rho c_p T$$
and we can then solve for the temperature.
But once the temperature reaches the melting point, additional heat is converted to the latent heat of melting of the material.

In [ ]:
ρ = Constant(1.0)
c_p = Constant(1.0)
T_m = Constant(0.0)
k = Constant(1.0)

def temperature(E):
    return firedrake.min_value(E / (ρ * c_p), T_m)

The fluxes of energy are from both advection and diffusion, and there are sources/sinks at both the top and bottom.
In terrain-following coordinates, the variational form of the problem is
$$\begin{align}
0 = & \int\left(h\frac{\partial E}{\partial t}\phi - hE\omega\frac{\partial\phi}{\partial\zeta} + h^{-1}k\frac{\partial T}{\partial\zeta}\frac{\partial\phi}{\partial\zeta} - hQ\phi\right)d\zeta \\
 & \qquad\qquad + \dot aE_{\dot a}\,\phi\Big|_{\zeta = 1} - \dot bE_{\dot b}\,\phi\Big|_{\zeta = 0}
\end{align}$$
for all test functions $\phi$.
Some of the inputs to this equation are dependent on each other, but for now we write it and the resulting code in the most general form possible.
For example, the vertical velocity $\omega$ is equal to the (normalized) ablation rate $h^{-1}\dot b$.

In [ ]:
def energy_form(**kwargs):
    field_names = (
        "energy",
        "thickness",
        "vertical_velocity",
        "conductivity",
        "accumulation",
        "surface_energy",
        "ablation",
        "basal_energy",
        "heat_source",
        "test_function"
    )
    E, h, ω, k, a, E_a, b, E_b, f_b, ϕ = map(kwargs.get, field_names)
    T = temperature(E)
    F_cells = (
        h * Dt(E) * ϕ -
        h * inner(E * ω, grad(ϕ)) +
        k / h * inner(grad(T), grad(ϕ))
    ) * dζ
    F_facets = a * E_a * ϕ * ds((2,)) + (f_b - b * E_b) * ϕ * ds((1,))
    return F_cells - F_facets

Now we need to make all the inputs.
First, we make the surface accumulation rate a periodic function of time:
$$a = a_0 + \delta a \sin(2\pi t / \tau)$$
where we choose the units of time so that a period is equal to $\tau = 1$.

In [ ]:
t = Constant(0.0)

a_0 = Constant(0.125)
δa = Constant(0.125)
a = a_0 + δa * firedrake.sin(2 * π * t)

We also need to decide what the energy density of the material accumulated at the surface is.
The melting temperature is at a surface energy density of 0.0, so we'll need a negative value.

In [ ]:
E_a = Constant(-1.0)

Next we need to determine the ablation rate at the base.
The ablation rate is determined by the temperature gradient in the material and the heat supplied at the base:
$$\dot b = \frac{k\nabla T + f_b}{\rho L}$$
It's worth piecing out the different scenarios for what the basal melt rate can do at different temperatures.
If the temperature is below the melting point, then no melting can occur.
If the ice base is at the melting point but the temperature gradient at the base is non-zero, then this conduction of heat into the column mitigates the total melt rate at the base.
Finally, if the ice base is at the melting point and the energy density is above the melting point, then there is some finite interval in which the temperature is all at $T_m$ and thus the temperature gradient is zero.
In that case all of the heat energy supplied at the base goes to melting.

Finally, we want the system to settle into some steady limit cycle eventually.
At the top of the domain, we're putting in mass at a rate of $\dot a$ at an energy density $E_{\dot a}$.
We need to supply enough heat to (1) raise this material to the melting point, and (2) overcome the latent heat and melt all of what's accumulated in a single cycle.
This means that the basal flux has to be equal to
$$f_b = (E_{\dot a} + \rho L)a_0$$
since the oscillatory term integrates to zero.

In [ ]:
L = Constant(1.0)
f_b = ρ * L * a_0

T = temperature(E)
b = firedrake.conditional(
    E < ρ * c_p * T_m,
    0.0,
    (k * T.dx(0) + f_b) / (ρ * L)
)

Finally, since we've assumed that there is no internal deformation or stretching of the material, the vertical velocity is equal to the melt rate.

In [ ]:
ω = -firedrake.as_vector((b / h,))

In [ ]:
E, h = firedrake.split(z)
ϕ, η = firedrake.TestFunctions(Z)

fields = {
    "energy": E,
    "thickness": h,
    "accumulation": a,
    "ablation": b,
    "vertical_velocity": ω,
    "conductivity": k,
    "heat_source": f_b,
    "surface_energy": E_a,
    "basal_energy": ρ * L,
}

F_energy = energy_form(**fields, test_function=ϕ)
F_thickness = thickness_form(**fields, test_function=η)
F = F_energy + F_thickness

In [ ]:
num_steps_per_year = 24
dt = Constant(1.0 / num_steps_per_year)
method = irksome.BackwardEuler()
solver = irksome.TimeStepper(F, method, t, dt, z)

In [ ]:
zs = [z.copy(deepcopy=True)]
final_time = 10.0
num_steps = int(final_time * num_steps_per_year)
for step in tqdm.trange(num_steps):
    solver.advance()
    t.assign(t + dt)
    zs.append(z.copy(deepcopy=True))

In [ ]:
E, h = z.subfunctions
E.dat.data_ro.min(), E.dat.data_ro.max()

In [ ]:
hs = np.array([float(z.subfunctions[1]) for z in zs])
hs.min(), hs.max()

In [ ]:
fig, ax = plt.subplots()
ax.plot(hs);